<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [1]:
models_compare = {
                    'skymodel1.txt': 'pybdsm-meerkat_SourceRecovery-casa.lsm.html',
                    'skymodel2.txt': 'pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html',
                    'skymodel3.txt': 'pybdsm-meerkat_SourceRecovery-lwimager.lsm.html',
                    'skymodel4.txt': 'pybdsm-meerkat_SourceRecovery-tclean.lsm.html',
                    'skymodel5.txt': 'pybdsm-meerkat_SourceRecovery-wsclean.lsm.html',
                 }
models_dr = {
            'pybdsm-meerkat_SourceRecovery-lwimager.lsm.html': 'meerkat_SourceRecovery_lwimager-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-wsclean.lsm.html': 'meerkat_SourceRecovery_wsclean-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html': 'meerkat_SourceRecovery_ddfacet-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-tclean.lsm.html': 'meerkat_SourceRecovery_tclean-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-casa.lsm.html': 'meerkat_SourceRecovery_casa-cube.residual.fits',
            }
RESIDUALS = [
             'meerkat_SourceRecovery_lwimager-cube.residual.fits',
             'meerkat_SourceRecovery_wsclean-cube.residual.fits',
             'meerkat_SourceRecovery_ddfacet-cube.residual.fits',
             'meerkat_SourceRecovery_tclean-cube.residual.fits',
             'meerkat_SourceRecovery_casa-cube.residual.fits',
            ]
res_noise_images = {'model': 'skymodel1.txt',
                    'meerkat_SourceRecovery_lwimager-cube.residual.fits': 'lwimager_noise.dirty.fits',
                    'meerkat_SourceRecovery_wsclean-cube.residual.fits': 'wsclean_noise-MFS-image.fits',
                    'meerkat_SourceRecovery_ddfacet-cube.residual.fits': 'ddfacet_noise.app.restored.fits',
                    'meerkat_SourceRecovery_tclean-cube.residual.fits': 'tclean_noise.image.dirty.fits',
                    'meerkat_SourceRecovery_casa-cube.residual.fits': 'casa_noise.dirty.fits',
                    }
#tolerance = 0.00002 # area factor around the source
#tolerance = 0.0001 # extended
tolerance = 0.00001 # point

In [2]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

directory = widgets.Text(
    value='input',
    placeholder='input',
    description='Input directory:',
    disabled=False
)

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)
display(directory)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description=u'Configure', style=ButtonStyle())

Text(value=u'input', description=u'Input directory:', placeholder=u'input')

In [3]:
import glob
import combat
import matplotlib
import numpy as np
from plotly import tools
from tabletext import to_text
import plotly.graph_objs as go
from plotly import offline as py
from collections import OrderedDict
from scipy.stats import norm, linregress
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
from aimfast.aimfast import residual_image_stats, image_dynamic_range, model_dynamic_range#, get_json_dict
py.init_notebook_mode(connected=True)

/tmp/venvs/drastic/local/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [4]:
reload(combat)
unit_scaler = 1000
ARCSEC_DEG = 1.0/(60*60)
results = combat.property_results(models=models_compare, tolerance=tolerance, input=directory.value)
PLOTS = len(models_compare)
PLOT_NUM = { 'colorbar': 
            {   #num of plots: [colorbar spacing, colorbar y, colorbar len]
                1: [0.90, 0, 0],
                2: [0.59, 0.78, 0.4],
                3: [0.41, 0.81, 0.34],
                4: [0.28, 0.86, 0.31],
                5: [0.207, 0.93, 0.2]
            }
           }

Loading input/skymodel1.txt: ASCII table


IOError: [Errno 2] No such file or directory: 'input/skymodel1.txt'

In [ ]:
def run_all(ev):
    # Runn all cells
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+31)'))

button = widgets.Button(
    description="PLOT"
)
button.on_click(run_all)
_scaley = widgets.Checkbox(
    value=False,
    description='Same Y-scale',
    disabled=False
)
_scalex = widgets.Checkbox(
    value=False,
    description='Same X-scale',
    disabled=False
)
data_range = widgets.IntSlider(
    value=100,
    min=1,
    max=100,
    step=1,
    description='Bin by flux(%):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
display(button)
display(_scaley)
display(_scalex)
display(data_range)

In [ ]:
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

## Photometric Measurements

In [14]:
table_data = [["Imager", "Slope", "R2-score", "RMS Error (mJy)", "Intercept (mJy)"]]
bg_color = 'rgb(229,229,229)'
bg_color = ''
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in sorted(models_compare.items()):
    counter+=1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    I_min_max += flux_out_data
    err_I_min_max += flux_out_err_data
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    reg = linregress(flux_in_data, flux_out_data)
    flux_R_score = reg.rvalue
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    table_data.append([heading.split('-')[-1].upper(), "{:.4f}".format(reg.slope),
                       "{:.4f}".format(flux_R_score),
                       " {:.4f}".format(np.sqrt(flux_MSE)*unit_scaler),
                       "{:.4f}".format(reg.intercept*unit_scaler)])
    counter += 1
print(to_text(table_data))
matplotlib.pylab.close()

┌──────────┬────────┬──────────┬─────────────────┬─────────────────┐
│ Imager   │ Slope  │ R2-score │ RMS Error (mJy) │ Intercept (mJy) │
├──────────┼────────┼──────────┼─────────────────┼─────────────────┤
│ CASA     │ 0.9380 │ 0.9961   │  0.1175         │ -0.0245         │
├──────────┼────────┼──────────┼─────────────────┼─────────────────┤
│ DDFACET  │ 0.9501 │ 0.9954   │  0.1056         │ -0.0249         │
├──────────┼────────┼──────────┼─────────────────┼─────────────────┤
│ LWIMAGER │ 0.9972 │ 0.9963   │  0.0606         │ -0.0299         │
├──────────┼────────┼──────────┼─────────────────┼─────────────────┤
│ TCLEAN   │ 0.9601 │ 0.9901   │  0.1674         │ -0.0961         │
├──────────┼────────┼──────────┼─────────────────┼─────────────────┤
│ WSCLEAN  │ 0.9345 │ 0.9964   │  0.1129         │ -0.0151         │
└──────────┴────────┴──────────┴─────────────────┴─────────────────┘


In [29]:
data = []
im_titles =[]
flux_im = dict()
for input_model, output_model in sorted(models_compare.items()):
    header = output_model[:-9]
    #im_titles.append('<b>{:s} flux density</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=1, shared_yaxes=False, print_grid=False,
                          vertical_spacing = 0.06, subplot_titles=sorted(im_titles))
j = 0
i = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
annotate = []
for input_model, output_model in sorted(models_compare.items()):
    i += 1
    counter += 1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    reg = linregress(flux_in_data, flux_out_data)
    flux_R_score = reg.rvalue
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    annotate.append(go.Annotation(
            x=0.0012*unit_scaler,
            y=flux_in_data[-1]*unit_scaler + 0.0005*unit_scaler,
            xref='x{:d}'.format(counter),
            yref='y{:d}'.format(counter),
            text="Slope: {:.4f} | Intercept: {:.4f} | RMS Error: {:.4f} | R2: {:.4f} ".format(
                reg.slope, reg.intercept*unit_scaler, np.sqrt(flux_MSE)*unit_scaler, flux_R_score),
            ax=0,
            ay=-40,
            showarrow=False,
            bordercolor='#c7c7c7',
            borderwidth=2,
            font=dict(color = "black", size = 12.5),
        ))
    fig.append_trace(go.Scatter(x=np.array([sorted(flux_in_data)[0], sorted(flux_in_data)[-1]])*unit_scaler,
                                showlegend=False,
                                marker = dict(color = 'rgb(0,0,255)'),
                                y=np.array([sorted(flux_in_data)[0], sorted(flux_in_data)[-1]])*unit_scaler,
                                mode = 'line'), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(flux_in_data)*unit_scaler, y=np.array(flux_out_data)*unit_scaler,
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % heading,
                                marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', titlefont=dict(size=16.5),
                                                              len=0.15, y=0.91-j)
                                             ),
                                error_y=dict(type='data', array=np.array(flux_out_err_data)*unit_scaler,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig['layout'].update(title='', height=2000, width=600,
                     #    paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(x=0.8,y=1.0),)
    fig['layout'].update(
        {'yaxis{}'.format(counter):YAxis(title=u'$I_{out} (mJy)$',gridcolor='rgb(255,255,255)',
        #range=[1,10],
        tickfont=dict(size=15),
        titlefont=dict(size=19),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter):XAxis(title=u'$I_{in} (mJy)$', #position=0.5,
                                                            titlefont=dict(size=19),
                                                            showgrid=True,
                                                            showline=True,
                                                            overlaying='x')})
    if counter == len(models_compare.keys()):
        fig['layout']['annotations'].extend(annotate)
    j+=PLOT_NUM['colorbar'][PLOTS][0]
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Astrometric Measurements

In [16]:
table_data = [["Imager", "RA mean", "DEC mean", "RA sigma", "DEC sigma", "Sources",
               "1-sigma", "2-sigma", "3-sigma", "Outliers"]]
counter = 0
pos_min_max = []
DEC_off_min_max = []
RA_off_min_max = []
for input_model, output_model in sorted(models_compare.items()):
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    recovered_sources = 0
    one_sigma_sources = 0
    two_sigma_sources = 0
    three_sigma_sources = 0
    rest_of_sources = 0
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
   
    RA_mean = np.mean(RA_offset)
    DEC_mean = np.mean(DEC_offset)
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    pi, cos, sin = np.pi, np.cos, np.sin
    theta = np.linspace(0,2*pi,len(DEC_offset))
    x1 = RA_mean+(r1*cos(theta))
    y1 = DEC_mean+(r2*sin(theta))
    x2 = RA_mean+(2*r1*cos(theta))
    y2 = DEC_mean+(2*r2*sin(theta))
    x3 = RA_mean+(3*r1*cos(theta))
    y3 = DEC_mean+(3*r2*sin(theta))
#    x2 = 2*RA_mean+(2*r1*cos(theta))
#    y2 = 2*DEC_mean+(2*r2*sin(theta))
#    x3 = 3*RA_mean+(3*r1*cos(theta))
#    y3 = 3*DEC_mean+(3*r2*sin(theta))
    recovered_sources = len(DEC_offset)
    one_sigma_sources = len([(ra_off, dec_off) for ra_off, dec_off in zip(RA_offset, DEC_offset)
                          if abs(ra_off) <= max(abs(x1)) and abs(dec_off) <= max(abs(y1))])
    two_sigma_sources = len([(ra_off, dec_off) for ra_off, dec_off in zip(RA_offset, DEC_offset)
                             if abs(ra_off) <= max(abs(x2)) and abs(dec_off) <= max(abs(y2))])
    two_sigma_sources = two_sigma_sources - one_sigma_sources
    three_sigma_sources = len([(ra_off, dec_off) for ra_off, dec_off in zip(RA_offset, DEC_offset)
                               if abs(ra_off) <= max(abs(x3)) and abs(dec_off) <= max(abs(y3))])
    three_sigma_sources = three_sigma_sources - (two_sigma_sources + one_sigma_sources)
    rest_of_sources = len([(ra_off, dec_off) for ra_off, dec_off in zip(RA_offset, DEC_offset)
                           if abs(ra_off) > max(abs(x3)) or abs(dec_off) > max(abs(y3))])
                      #     if all(abs(ra_off) > abs(x) for x in x3) or all(abs(dec_off) > abs(y) for y in y3)])
    table_data.append([heading.split('-')[-1].upper(), "{:.4f}".format(RA_mean), "{:.4f}".format(DEC_mean),
                       "{:.4f}".format(r1), "{:.4f}".format(r2), recovered_sources,
                       one_sigma_sources, two_sigma_sources, three_sigma_sources, rest_of_sources])
print(to_text(table_data))

┌──────────┬─────────┬──────────┬──────────┬───────────┬─────────┬─────────┬─────────┬─────────┬──────────┐
│ Imager   │ RA mean │ DEC mean │ RA sigma │ DEC sigma │ Sources │ 1-sigma │ 2-sigma │ 3-sigma │ Outliers │
├──────────┼─────────┼──────────┼──────────┼───────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ CASA     │ 0.0664  │ 0.0648   │ 0.0791   │ 0.0757    │      97 │      82 │       4 │       6 │        5 │
├──────────┼─────────┼──────────┼──────────┼───────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ DDFACET  │ 0.0599  │ 0.0687   │ 0.0771   │ 0.0844    │      97 │      79 │       7 │       6 │        5 │
├──────────┼─────────┼──────────┼──────────┼───────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ LWIMAGER │ 0.0983  │ 0.0996   │ 0.0982   │ 0.1206    │      97 │      81 │      10 │       2 │        4 │
├──────────┼─────────┼──────────┼──────────┼───────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ TCLEAN   │ 0.1909  │ 0.461

In [27]:
data = []
im_titles =[]
for input_model, output_model in sorted(models_compare.items()):
    header = output_model[:-9]
 #   im_titles.append('<b>{:s} Position</b>'.format(header.upper()))
 #   im_titles.append('<b>{:s} Position</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=sorted(im_titles))
j = 0
i = -1
counter = 0
for input_model, output_model in sorted(models_compare.items()):
    i += 1
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[-2]))
    fig.append_trace(go.Scatter(x=np.array(flux_in_data)*unit_scaler, y=np.array(delta_pos_data),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Jet', reversescale=True,
                                              colorbar = dict(title='Phase center dist (arcsec)',
                                                              titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j,
                                                              x=0.39)
                                             ),
                                error_y=dict(type='data', #array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(RA_offset), y=np.array(DEC_offset),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(heading),
                                marker = dict(color = np.array(flux_out_data)*unit_scaler, showscale=True,
                                              colorscale='Viridis',
                                              reversescale=True,
                                              colorbar = dict(title='Output flux (mJy)',
                                                              titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    RA_mean = np.mean(RA_offset)
    DEC_mean = np.mean(DEC_offset)
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    pi, cos, sin = np.pi, np.cos, np.sin
    theta = np.linspace(0,2*pi,len(DEC_offset))
    x1 = RA_mean+(r1*cos(theta))
    y1 = DEC_mean+(r2*sin(theta))
    x2 = RA_mean+(2*r1*cos(theta))
    y2 = DEC_mean+(2*r2*sin(theta))
    x3 = RA_mean+(3*r1*cos(theta))
    y3 = DEC_mean+(3*r2*sin(theta))
    fig.append_trace(go.Scatter(x=x1, y=y1,
                                mode = 'lines', showlegend=False,#True if i == 0 else False,
                                name = '1σ',
                                text = '1σ ~ {:f}'.format(np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(0, 0, 255)')), i+1, 2)
#    fig.append_trace(go.Scatter(x=x2, y=y2,
#                                mode = 'lines', showlegend=True if i == 0 else False,
#                                name = '2σ',
#                                text = '2σ ~ {:f}'.format(2*np.sqrt(r1*r2)),
#                                marker = dict(color = 'rgb(255, 0, 0)')), i+1, 2)
#    fig.append_trace(go.Scatter(x=x3, y=y3,
#                                mode = 'lines', showlegend=True if i == 0 else False,
#                                name = '3σ',
#                                text = '3σ ~ {:f}'.format(3*np.sqrt(r1*r2)),
#                                marker = dict(color = 'rgb(255, 255, 0)')), i+1, 2)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(xanchor=True, x=1.2,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Delta position [arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #                                   range=axis_min_max if _scaley.value else [],
        # range=[-0.1,3.7], # extended
        range=[-0.1,1.5], # point
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Dec offset [arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
        #                                     range=dec_min_max if _scaley.value else [],
        #  # range=[-0.1,3.7], # extended
        range=[-0.5,1.3], # point
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title=u'$I_{in} (mJy)$',
                                                            titlefont=dict(size=17),
                                                            zeroline=True, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='RA offset [arcsec]',
                                                              titlefont=dict(size=17),
                                                              #range=ra_min_max if _scalex.value else [],
                                                              #range=[-1.5, 3.2],
                                                              range=[-0.1, 0.7],
                                                              zeroline=False)})#domain=[0.505, 0.8])})
    fig['layout']['annotations'].update({ 'font':{'size': 10}})
    j+=PLOT_NUM['colorbar'][PLOTS][0]
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Morphological Measurements

In [18]:
maj_table_data = [["Major-Axis / Imager", "Gradient", "R-sqaured", "RMS Error (arcsec)", "Intercept (arcsec)", "Unresolved"]]
min_table_data = [["Minor-Axis / Imager", "Gradient", "R-sqaured", "RMS Error (arcsec)", "Intercept (arcsec)", "Unresolved"]]
j = 0
i = -1
counter = 0
for input_model, output_model in sorted(models_compare.items()):
    i += 1
    counter+=1
    SCALE = []
    SCALE_ERR = []
    flux_in_data = []
    DELTA_PHASE0 = []
    source_labels = []
    MAJ_MIN_angle_in = []
    MAJ_MIN_angle_out = []
    unresolved = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        if results[heading]['shape'][n][0][0] != 0.0:
            MAJ_MIN_angle_out.append(results[heading]['shape'][n][0])
            MAJ_MIN_angle_in.append(results[heading]['shape'][n][2])
            SCALE.append(results[heading]['shape'][n][3])
            SCALE_ERR.append(results[heading]['shape'][n][4])
            flux_in_data.append(results[heading]['shape'][n][5])
            source_labels.append(results[heading]['shape'][n][6])
            DELTA_PHASE0.append(results[heading]['position'][n][-3])
        else:
            unresolved.append(results[heading]['shape'][n][0][0])
    if MAJ_MIN_angle_out:
        zipped_props = zip(MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR)
        MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR = zip(
            *sorted(zipped_props, key=lambda x: x[0]))
        maj_in = [maj_min_angle_in[0] for maj_min_angle_in in MAJ_MIN_angle_in]
        maj_out = [maj_min_angle_out[0] for maj_min_angle_out in MAJ_MIN_angle_out]
        min_in = [maj_min_angle_in[1] for maj_min_angle_in in MAJ_MIN_angle_in]
        min_out = [maj_min_angle_out[1] for maj_min_angle_out in MAJ_MIN_angle_out]
        angle_offset = [(maj_min_angle_out[2] - maj_min_angle_in[2]) for maj_min_angle_out, maj_min_angle_in in zip(
            MAJ_MIN_angle_out, MAJ_MIN_angle_in)]
        maj_MSE = mean_squared_error(maj_in, maj_out)
        reg = linregress(maj_in, maj_out)
        maj_R_score = reg.rvalue
        maj_table_data.append([heading.split('-')[-1].upper(), "{:.4f}".format(reg.slope),
                       "{:.4f}".format(maj_R_score),
                       " {:.4f}".format(np.sqrt(maj_MSE)),
                       "{:.4f}".format(reg.intercept),
                        len(unresolved)])
        min_MSE = mean_squared_error(min_in, min_out)
        reg = linregress(min_in, min_out)
        min_R_score = reg.rvalue
        min_table_data.append([heading.split('-')[-1].upper(), "{:.4f}".format(reg.slope),
                       "{:.4f}".format(min_R_score),
                       " {:.4f}".format(np.sqrt(min_MSE)),
                       "{:.4f}".format(reg.intercept),
                        len(unresolved)])
print(to_text(maj_table_data))
print(to_text(min_table_data))

┌─────────────────────┬──────────┬───────────┬────────────────────┬────────────────────┬────────────┐
│ Major-Axis / Imager │ Gradient │ R-sqaured │ RMS Error (arcsec) │ Intercept (arcsec) │ Unresolved │
├─────────────────────┼──────────┼───────────┼────────────────────┼────────────────────┼────────────┤
│ TCLEAN              │ nan      │ 0.0000    │  0.0000            │ nan                │         35 │
└─────────────────────┴──────────┴───────────┴────────────────────┴────────────────────┴────────────┘
┌─────────────────────┬──────────┬───────────┬────────────────────┬────────────────────┬────────────┐
│ Minor-Axis / Imager │ Gradient │ R-sqaured │ RMS Error (arcsec) │ Intercept (arcsec) │ Unresolved │
├─────────────────────┼──────────┼───────────┼────────────────────┼────────────────────┼────────────┤
│ TCLEAN              │ nan      │ 0.0000    │  0.0000            │ nan                │         35 │
└─────────────────────┴──────────┴───────────┴────────────────────┴───────────────

/home/athanaseus/.local/lib/python2.7/site-packages/scipy/stats/_stats_mstats_common.py:107: RuntimeWarning:

invalid value encountered in double_scalars

/home/athanaseus/.local/lib/python2.7/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning:

divide by zero encountered in double_scalars



In [19]:
data = []
im_titles =[]
for input_model, output_model in sorted(models_compare.items()):
    header = output_model[:-9]
#    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))
#    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=sorted(im_titles))
j = 0
i = -1
counter = 0
for input_model, output_model in sorted(models_compare.items()):
    table_data = []
    i += 1
    counter+=1
    SCALE = []
    SCALE_ERR = []
    flux_in_data = []
    DELTA_PHASE0 = []
    source_labels = []
    MAJ_MIN_angle_in = []
    MAJ_MIN_angle_out = []
    unresolved = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        MAJ_MIN_angle_out.append(results[heading]['shape'][n][0])
        MAJ_MIN_angle_in.append(results[heading]['shape'][n][2])
        SCALE.append(results[heading]['shape'][n][3])
        SCALE_ERR.append(results[heading]['shape'][n][4])
        flux_in_data.append(results[heading]['shape'][n][5])
        source_labels.append(results[heading]['shape'][n][6])
        DELTA_PHASE0.append(results[heading]['position'][n][-3])
    try:
        zipped_props = zip(MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR)
        MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR = zip(
            *sorted(zipped_props, key=lambda x: x[0]))
        maj_in = [maj_min_angle_in[0] for maj_min_angle_in in MAJ_MIN_angle_in]
        maj_out = [maj_min_angle_out[0] for maj_min_angle_out in MAJ_MIN_angle_out]
        min_in = [maj_min_angle_in[1] for maj_min_angle_in in MAJ_MIN_angle_in]
        min_out = [maj_min_angle_out[1] for maj_min_angle_out in MAJ_MIN_angle_out]
        angle_offset = [(maj_min_angle_out[2] - maj_min_angle_in[2]) for maj_min_angle_out, maj_min_angle_in in zip(
            MAJ_MIN_angle_out, MAJ_MIN_angle_in)]
        flux_MSE = mean_squared_error(maj_in, maj_out)
        reg = linregress(maj_in, maj_out)
        flux_R_score = reg.rvalue
        fig.append_trace(go.Scatter(x=np.array([sorted(maj_in)[0], sorted(maj_in)[-1]]),
                                    showlegend=False,
                                    marker = dict(color = 'rgb(0,0,255)'),
                                    y=np.array([sorted(maj_in)[0], sorted(maj_in)[-1]]),
                                    mode = 'line'), i+1, 1)
        fig.append_trace(go.Scatter(x=np.array([sorted(min_in)[0], sorted(min_in)[-1]]),
                                    showlegend=False,
                                    marker = dict(color = 'rgb(0,0,255)'),
                                    y=np.array([sorted(min_in)[0], sorted(min_in)[-1]]),
                                    mode = 'line'), i+1, 2)
    except ValueError:
        pass
    fig.append_trace(go.Scatter(x=maj_in, y=maj_out, mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = np.array(flux_out_data)*unit_scaler, showscale=True, colorscale='Jet',
                                              reversescale=True,
                                              colorbar = dict(title='Output flux (mJy)',
                                                              titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j,
                                                              x=0.39)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=min_in, y=min_out,
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = np.array(flux_out_data)*unit_scaler, showscale=True,
                                              colorscale='Jet', reversescale=True,
                                                  colorbar = dict(title='Output flux (mJy)',
                                                              titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(xanchor=True)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Output maj axis[arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        range=[-1,13],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Ouput min axis[arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
        range=[-1,13],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input maj axis[arcsec]',
                                                            titlefont=dict(size=17),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Input min axis[arcsec]',
                                                              titlefont=dict(size=17),
                                                              zeroline=False)})# domain=[0.505, 0.8])}
    fig['layout']['annotations'].update({'font':{'size': 12}})
    j+=PLOT_NUM['colorbar'][PLOTS][0]
py.iplot(fig, filename='make-subplots-multiple-with-titles')

/home/athanaseus/.local/lib/python2.7/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning:

invalid value encountered in double_scalars



## Spectral Index Measurements

In [20]:
bin_colors = {1: '#f0f8ff',
              2: '#dcdcdc',
              3: '#ffe4c4',
              4: '#ff7f50',
              5: '#7fffd4'
             }
flux_bins = widgets.IntSlider(
    value=5,
    min=1,
    max=100,
    step=1,
    description='Bin by flux:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
spi_bins = widgets.IntSlider(
    value=5,
    min=1,
    max=100,
    step=1,
    description='Bin by spi:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
y_axis_range = widgets.IntRangeSlider(
    value=[-80, 80],
    min=-150,
    max=150,
    step=1,
    description='Y-axis limit',
    disabled=False,
    units= 'arcsec',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
display(flux_bins)
display(spi_bins)
display(y_axis_range)

IntSlider(value=5, continuous_update=False, description=u'Bin by flux:', min=1)

IntSlider(value=5, continuous_update=False, description=u'Bin by spi:', min=1)

IntRangeSlider(value=(-80, 80), continuous_update=False, description=u'Y-axis limit', max=150, min=-150)

In [21]:
import matplotlib
import pylab as plt
from scipy.stats import norm
fig = plt.figure()
num_bins = flux_bins.value
i = -1
j = 0
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
#========================
#MEAN = " Mean"
#MEDIAN = " Median"
#MODE = " Mode"
#STD = " Standard deviation"
#========================
global_table_data = [["Global", "Mean", "Median", "Mode", "Standard Deviation"]]
if results[models_compare.values()[0][:-9]]['spi']:
    for input_model, output_model in sorted(models_compare.items()):
        header = output_model[:-9]
        bin_table_data = [[header.split('-')[-1].upper(), "Mean", "Median", "Mode", "Standard Deviation"]]
        i += 1
        counter+=1
        I_in = []
        name_labels = []
        spi_in_data = []
        spi_out_data = []
        spi_err_data = []
        dist_from_phase = []
        num_data_points = len(results[header]['spi'])
        for n in range(num_data_points):
            spi_out_data.append(results[header]['spi'][n][0])
            spi_err_data.append(results[header]['spi'][n][1])
            spi_in_data.append(results[header]['spi'][n][2])
            dist_from_phase.append(results[header]['spi'][n][3])
            I_in.append(results[header]['spi'][n][4])
            name_labels.append(results[header]['spi'][n][5])
        zipped_props = zip(I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels)
        (I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels) = zip(
            *sorted(zipped_props, key=lambda x: x[0]))
        spi_R_score = r2_score(spi_in_data, spi_out_data)
        spi_MSE = mean_squared_error(spi_in_data, spi_out_data)
        spi_out_in = [float(spi_out)/spi_in for spi_out,spi_in in zip(spi_out_data,spi_in_data)]
        ranger = num_data_points/num_bins
        start, end = [-ranger, 0]
        if num_bins > 1:
            for b in range(num_bins):
                end += ranger
                start += ranger
                spi_out_data_range = spi_out_data[start:num_data_points if (b + 1) == num_bins else end],
                n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                                          spi_bins.value, normed=False, facecolor='blue',
                                                          orientation='horizontal', alpha=0.5)
                (mu, sigma) = norm.fit(spi_out_data_range)
                median = np.median(spi_out_data_range)
                peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
                bin_table_data.append(["BIN-{:d}".format(b+1), "{:4f}".format(mu), "{:4f}".format(median),
                                      "{:4f}".format(peak), "{:4f}".format(sigma)])
                
                
                #IM = header.split('-')[-1].upper()
                #if b == 4:
                #    MEAN += " & {:.4f}".format(mu)
                #    MEDIAN += " & {:.4f}".format(median)
                #    MODE += " & {:.4f}".format(peak)
                #    STD += " & {:.4f}".format(sigma)
                
                
        print(to_text(bin_table_data))
        n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                                  spi_bins.value, normed=False, facecolor='blue',
                                                  orientation='horizontal', alpha=0.5)
        (mu, sigma) = norm.fit(spi_out_data)
        median = np.median(spi_out_data)
        peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
        global_table_data.append([header.split('-')[-1].upper(), "{:4f}".format(mu), "{:4f}".format(median),
                                "{:4f}".format(peak), "{:4f}".format(sigma)])
        counter += 1
        
        
#        MEAN += " & {:.4f}".format(mu)
#        MEDIAN += " & {:.4f}".format(median)
#        MODE += " & {:.4f}".format(peak)
#        STD += " & {:.4f}".format(sigma)
        
    plt.close(fig)
print(to_text(global_table_data))


#print MEAN + r"\\"
#print " \hline"
#print MEDIAN + r"\\"
#print " \hline"
#print MODE + r"\\"
#print " \hline"
#print STD + r"\\"
#print " \hline\hline"

KeyError: 'spi'

<Figure size 432x288 with 0 Axes>

In [ ]:
data = []
im_titles =[]
y_min_max = y_axis_range.value
y_ran_pos = [y_min_max[-1], y_min_max[-1]]
y_ran_neg = [y_min_max[0], y_min_max[0]]
flux_im = dict()
num_bins = flux_bins.value
for input_model, output_model in sorted(models_compare.items()):
    header = output_model[:-9]
    #im_titles.append('<b>{:s} spi measure</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,
                          subplot_titles=sorted(im_titles))
i = -1
j = 0
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
if results[header]['spi']:
#    for input_model, output_model in {'skymodel1.txt': 'pybdsm-meerkat_SourceRecovery-casa.lsm.html'}.items():
    for input_model, output_model in sorted(models_compare.items()):
        header = output_model[:-9]
        i += 1
        counter+=1
        I_in = []
        name_labels = []
        spi_in_data = []
        spi_out_data = []
        spi_err_data = []
        dist_from_phase = []
        num_data_points = len(results[header]['spi'])
        for n in range(num_data_points):
            spi_out_data.append(results[header]['spi'][n][0])
            spi_err_data.append(results[header]['spi'][n][1])
            spi_in_data.append(results[header]['spi'][n][2])
            dist_from_phase.append(results[header]['spi'][n][3])
            I_in.append(results[header]['spi'][n][4])
            name_labels.append(results[header]['spi'][n][5])
        zipped_props = zip(I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels)
        (I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels) = zip(
            *sorted(zipped_props, key=lambda x: x[0]))
        spi_R_score = r2_score(spi_in_data, spi_out_data)
        spi_MSE = mean_squared_error(spi_in_data, spi_out_data)
        spi_out_in = [float(spi_out)/spi_in for spi_out,spi_in in zip(spi_out_data,spi_in_data)]
#===========================================================================================================
        ranger = num_data_points/num_bins
        start, end = [-ranger, 0]
        for b in range(num_bins):
            end += ranger
            start += ranger
            fig.append_trace(go.Scatter(
                             x=[sorted(np.array(I_in)[start:num_data_points
                                                      if (b + 1) == num_bins else end]*unit_scaler)[0],
                                sorted(np.array(I_in)[start:num_data_points
                                                      if (b + 1) == num_bins else end]*unit_scaler)[-1]],
                             y=y_ran_pos,
                             showlegend=False,
                             mode= 'none',
                             fillcolor = bin_colors[b+1],
                             fill='tozeroy'), i+1, 1)
            fig.append_trace(go.Scatter(
                             x=[sorted(np.array(I_in)[start:num_data_points
                                                      if (b + 1) == num_bins else end]*unit_scaler)[0],
                                sorted(np.array(I_in)[start:num_data_points
                                                      if (b + 1) == num_bins else end]*unit_scaler)[-1]],
                             y=y_ran_neg,
                             showlegend=False,
                             mode= 'none',
                             fillcolor = bin_colors[b+1],
                             fill='tozeroy'), i+1, 1)
#===========================================================================================================
        fig.append_trace(go.Scatter(x=np.array([sorted(I_in)[0], sorted(I_in)[-1]])*1000, showlegend=False,
                                    marker = dict(color = 'rgb(0,0,255)'),
                                    y=np.array([-.7,-.7]), mode = 'line'), i+1, 1)
        fig.append_trace(go.Scatter(x=np.array(I_in)*1000, y=np.array(spi_out_data),
                                    mode = 'markers', showlegend=False,
                                    text = name_labels, name = '%s flux_ratio' % heading,
                                    marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                                  reversescale=False,
                                                  colorbar = dict(title='PTC distance (arcsec)',
                                                                   titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j)
                                                 ),
                                    error_y=dict(type='data', array=spi_err_data,
                                                 color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
        fig['layout'].update(title='', height=1200, width=900,
                             paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                             legend=dict(x=0.8,y=1.0),)
        fig['layout'].update(
            {'yaxis{}'.format(counter):YAxis(title=u'$SPI_{out}$',gridcolor='rgb(255,255,255)',
            range=y_min_max,
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=True)})
        fig['layout'].update({'xaxis{}'.format(counter):XAxis(title='$I_{in} (mJy)$', position=0.0,
                                                                titlefont=dict(size=17),
                                                                overlaying='x')})
        fig['layout']['annotations'].update({ 'font':{'size': 10}})
        j+=PLOT_NUM['colorbar'][PLOTS][0]
    py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Dynamic Ranges

In [ ]:
beam_factor = 4
DRs = OrderedDict()
im_titles = []
dir = directory.value
for m, r in models_dr.items():
    DRs[r[23:-19].upper()] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 6, beam_factor)
#    DRs[r[:-19]] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 94, beam_factor)
zipped_props = zip(DRs.keys(), DRs.values())
ims, values = zip(*sorted(zipped_props, key=lambda x: x[0]))
drs, names = [], []
for val in values:
    drs.append(val[0])
    names.append("DR=({:f},{:f})".format(val[1], val[-1]))
fig = tools.make_subplots(rows=1, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15)
fig.append_trace(go.Scatter(x=ims, showlegend=False, text = names,
                            y=np.array(drs), mode = 'markers', marker=dict(size='30')), 1, 1)
#fig['layout'].update(title='<b>SNR with self-cal iterations</b>', height=900, width=900,
fig['layout'].update(title='', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                     legend=dict(x=0.8,y=1.0))
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Dynamic Range',gridcolor='rgb(255,255,255)',
        tickfont=dict(size=25),
        titlefont=dict(size=30),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title='Images', position=0.0,
                                                        tickfont=dict(size=20),
                                                        titlefont=dict(size=30),
                                                        showline=True,
                                                        overlaying='x')})
fig['layout']['titlefont'].update({'size':20})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Residual Statistics

In [ ]:
Res = OrderedDict()
for res in RESIDUALS:
    Res[res] = residual_image_stats('{:s}/{:s}'.format(directory.value, res), test_model='shapiro')

In [ ]:
names = []
mean = []
std = []
skew = []
kurt = []
normtest = []
table_data = [["Imager", "Mean (mJy/beam)", "STD (mJy/beam)", "Skewness", "Kurtosis", "Normality"]]
for name, stats in sorted(Res.items()):
    names.append(name[23:-19].upper())
    mean.append(stats['MEAN'])
    std.append(stats['STDDev'])
    skew.append(stats['SKEW'])
    kurt.append(stats['KURT'])
    normtest.append(stats['NORM'][0])
    table_data.append([name.split('.')[0].split('_')[-1], "{:.3E}".format(stats['MEAN']*unit_scaler),
                       "{:.3E}".format(stats['STDDev']), "{:.3E}".format(stats['SKEW']),
                       "{:.3f}".format(stats['KURT']), "{:.3f}".format(stats['NORM'][0])])
zipped_props = zip(names, mean, std, skew, kurt, normtest)
names, mean, std, skew, kurt, normtest = zip(*sorted(zipped_props, key=lambda x: x[0]))
print(to_text(table_data))

In [ ]:
im_titles = [
               'MEAN',
               'STANDARD DEVIATION',
               'SKEWNESS',
               'KURTOSIS',
               'NORMALITY'
            ]
    
fig = tools.make_subplots(rows=4, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,)
                    #      subplot_titles=im_titles)

x_axis_title = 'Residual Images'
#x_axis_title = 'Iterations'
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(mean), mode = 'marker'), 1, 1)
fig['layout'].update(title='', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(x=0.8,y=1.0),)
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Residual mean (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=20),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title='', position=0.0,
                                                tickfont=dict(size=17),
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(std), mode = 'marker'), 2, 1)
fig['layout'].update(
    {'yaxis{}'.format(2):YAxis(title=u'Residual sigma (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=20),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(2):XAxis(title='', position=0.0,
                                                tickfont=dict(size=17),
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(skew)*unit_scaler, mode = 'marker'), 3, 1)
fig['layout'].update(
    {'yaxis{}'.format(3):YAxis(title=u'Residual skewness (mJy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=20),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            dtick='',
            tickformat='',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(3):XAxis(title='', position=0.0,
                                                tickfont=dict(size=17),
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(kurt), mode = 'marker'), 4, 1)
fig['layout'].update(
    {'yaxis{}'.format(4):YAxis(title=u'Residual kurtosis',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=20),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(4):XAxis(title=x_axis_title, position=0.0,
                                                tickfont=dict(size=17),
                                                titlefont=dict(size=20),
                                                showline=True,
                                                overlaying='x')})
#fig.append_trace(go.Scatter(x=names, showlegend=False,
#                            y=np.array(normtest), mode = 'marker'), 5, 1)
#fig['layout'].update(
#    {'yaxis{}'.format(5):YAxis(title=u'Residual normality',gridcolor='rgb(255,255,255)',
#            tickfont=dict(size=15),
#            titlefont=dict(size=17),
#            showgrid=True,
#            showline=True,
#            showticklabels=True,
#            tickcolor='rgb(51,153,225)',
#            ticks='outside',
#            zeroline=False)})
#fig['layout'].update({'xaxis{}'.format(5):XAxis(title=x_axis_title, position=0.0,
#                                                titlefont=dict(size=17),
#                                                showline=True,
#                                                overlaying='x')})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

#### Source residual-to-noise ration measurements

In [ ]:
reload(combat)
try:
    skymodel = res_noise_images.pop('model')
except KeyError:
    pass
res = combat.res_noise_ratio(skymodel, res_noise_images, directory=directory.value)

In [ ]:
data = []
im_titles =[]
TO_MICRO = 1E6

for residual_image, noise_image in res_noise_images.items():
    header1 = residual_image[:-5][23:]
    header2 = noise_image[:-17][:]
    #im_titles.append('<b>{:s} RMS</b>'.format(header1.upper()))
    #im_titles.append('<b>{:s} residual-noise</b>'.format(header2.upper()))

fig = tools.make_subplots(rows=len(res_noise_images), cols=2, shared_yaxes=False, print_grid=False,
                          #horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
counter = 0
i=0
j=0
for residual_image, noise_image in res_noise_images.items():
    counter+=1
    SNR = []
    dist_from_phase = []
    src_list = []
    rmss = []
    residuals = []
    res_noise_ratio = []
    res_mean_noise_rms_ratio = []
    imager = header = residual_image
    k = 0
    for res_src in res[imager]:
        src_list.append(res_src[0])
        residuals.append(res_src[2])
        rmss.append(res_src[3])
        res_noise_ratio.append(res_src[4])
        res_mean_noise_rms_ratio.append(res_src[8])
        dist_from_phase.append(res_src[-1])
        k += 1
#    for n in range(len(results[imager]['spi'])):
#        SCALE.append(results[imager]['shape'][n][4])
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(rmss)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'noise',
                                marker = dict(color = 'rgb(255,0,0)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(residuals)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'residual',
                                marker = dict(color = 'rgb(0,0,255)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(res_noise_ratio),
                                mode = 'markers', showlegend=False,
                                text = name_labels,# name = '%s flux_ratio' % imager,
                                marker = dict(color = dist_from_phase, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Phase center dist (arcsec)',
                                                              titleside ='right',
                                                              len=PLOT_NUM['colorbar'][PLOTS][2],
                                                              y=PLOT_NUM['colorbar'][PLOTS][1]-j)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.15,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'rms [μJy]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title=u'$I_{res}/I_{noise}$',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=res_axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Sources',
                                                            titlefont=dict(size=17),
                                                            showline=True,
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Sources',
                                                              titlefont=dict(size=17),
                                                              showline=True,
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )
    i+=1
    j+=PLOT_NUM['colorbar'][PLOTS][0]
py.iplot(fig, filename='make-subplots-multiple-with-titles')

#### Random source residual-to-noise ration measurements

In [ ]:
reload(combat)
try:
    skymodel = res_noise_images.pop('model')
except KeyError:
    pass
res = combat.random_res_noise_ratio(res_noise_images, directory=directory.value,
                                    num_pix=4096, pix_size=1.0, num_areas=200)

In [ ]:
data = []
im_titles =[]
TO_MICRO = 1E6

for residual_image, noise_image in res_noise_images.items():
    header1 = residual_image[:-5][23:]
    header2 = noise_image[:-17][:]
   # im_titles.append('<b>{:s} RMS</b>'.format(header1.upper()))
    #im_titles.append('<b>{:s} residual-noise</b>'.format(header2.upper()))

fig = tools.make_subplots(rows=len(res_noise_images), cols=2, shared_yaxes=False, print_grid=False,
                          #horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i=0
j=0
counter = 0
for residual_image, noise_image in res_noise_images.items():
    counter+=1
    rmss = []
    residuals = []
    res_noise_ratio = []
    imager = residual_image
    k = 0
    for res_src in res[imager]:
        residuals.append(res_src[0])
        rmss.append(res_src[1])
        res_noise_ratio.append(res_src[2])
        k += 1
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(rmss)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'noise',
                                marker = dict(color = 'rgb(255,0,0)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(residuals)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'residual',
                                marker = dict(color='rgb(0,0,255)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(res_noise_ratio),
                                mode = 'markers', showlegend=False,
                                text = name_labels,# name = '%s flux_ratio' % imager,
                                 marker = dict(color='rgb(158, 63, 221)'),
                                error_y=dict(type='data',
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.0,y=1.05)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'rms [μJy]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title=u'$I_{res}/I_{noise}$',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=res_axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Sources',
                                                            titlefont=dict(size=17),
                                                            showline=True,
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Sources',
                                                              titlefont=dict(size=17),
                                                              showline=True,
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )
    i+=1
    j+=0.285
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Profiling of Algorithms

In [ ]:
path = directory.value
######################################
import json
path = '/home/athanaseus/Desktop'
def get_json_dict(filename):
    with open(filename) as data_file:
        data  = json.load(data_file)
        return data
#######################################
time_data, proc_names, trace_data = [], [], []
prof_files = [
#   'profiler.json',
#    'profiler_6h_extended.json',
   'profiler_6h_point.json'
]
colour = ['rgb(0,0,225)', 'rgb(225,0,0)']
plot_order = {
              0:'casa_clean',
              1:'ddfacet',
              2:'lwimager',
              3:'tclean',
              4:'wsclean'
             }
for c, prof_file in enumerate(prof_files):
    time_data, proc_names = [], []
    for i, proc in plot_order.items():
        t = prof_file.split('_')[1].split('.')[0]
        #prof_data = helper_module.get_json_dict('%s/%s' % (path, prof_file))
        prof_data = get_json_dict('%s/%s' % (path, prof_file))
        for proc_d, time_d in prof_data.items():
            if proc in proc_d:
                proc_names.append(proc_d.split('_%s_32MHz' % t)[0].split('image_')[-1].upper())
                time_data.append(time_d)
                
    trace = go.Scatter(x = proc_names, y = np.array(time_data), mode = 'lines+markers', showlegend=False,
                       marker=dict(color=colour[c]))
    trace_data.append(trace)
layout = go.Layout(
#    title='Radio Interferometry Algorithm Profiling',
    titlefont=dict(size=16),
    yaxis=dict(
        title='Time (s)',
        titlefont=dict(
            size=18,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    xaxis=dict(
        title='Imagers',
        titlefont=dict(size=18)
    ),
   # plot_bgcolor='rgb(229,229,229)'
)

fig = go.Figure(data=trace_data, layout=layout)#, layout=layout)
py.iplot(fig, filename='basic-scatters')

## View Reconstructed Maps

In [ ]:
%%html
<iframe src="http://js9.si.edu" width=990 height=740></iframe>

## Source meta-data

In [ ]:
src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(src_nm)

In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)